In [2]:
import sys
sys.path.append("/home/mariacst/exoplanets/running/.env/lib/python3.6/site-packages")
sys.path.append("../python/")
import emcee
import numpy as np
from scipy.interpolate import griddata
from derivatives import derivativeTint_wrt_A, derivativeTint_wrt_M
import imp
import mock_generation
imp.reload(mock_generation)
from mock_generation import mock_population_all
from astropy.constants import R_jup
import glob
import pickle
#import time
#from emcee_functions import lnprob

# Timing the script
import time


# Constant parameters & conversions ==========================================
rho0                    = 0.42 # Local DM density [GeV/cm3]
epsilon                 = 1.
Rsun                    = 8.178 # Sun galactocentric distance [kpc]
# ============================================================================
# Input parameters
#ex         = sys.argv[1]
ex = "fixedT10v100"
#rank       = int(sys.argv[2])
rank = 1
#nBDs       = int(sys.argv[3])
nBDs = 100000
relTobs    = 0.1
#sigma      = float(sys.argv[4])
sigma = 0.1
f_true     = 1.
#gamma_true = float(sys.argv[5])
gamma_true = 0.6
#rs_true    = float(sys.argv[6])
rs_true = 5
v          = 100.
# ------------------------------------------------------------------------
# Load theoretical cooling model
path = "/home/mariacst/exoplanets/running/data/"
data = np.genfromtxt(path + "./ATMO_CEQ_vega_MIRI.txt", unpack=True)
points = np.transpose(data[0:2, :])
values = data[2]

start = time.time()
# Mock observation
#np.random.seed(rank)
(robs, sigmarobs, Tobs, sigmaTobs, Mobs,
     sigmaMobs, Aobs, sigmaAobs) = mock_population_all(nBDs, relTobs, sigma,
                                      sigma, sigma, f_true, gamma_true,
                                      rs_true, rho0_true=rho0, v=v)

elapsed = time.time() - start
print("Mock observations took " + str(elapsed))
                                      
# Calculate derivatives Tint wrt Age and Mass
dervTint_A = np.ones(nBDs)
dervTint_M = np.ones(nBDs)
size       = 7000
h          = 0.001

# for i in range(nBDs):
#     dervTint_A[i] = derivativeTint_wrt_A(Mobs[i], Aobs[i], points, values,
#                                          size=size, h=h)
#     dervTint_M[i] = derivativeTint_wrt_M(Mobs[i], Aobs[i], points, values,
#                                          size=size, h=h)

# Path for 10^5 derivatives
path_der = "/hdfs/local/mariacst/exoplanets/data_der/"

steps = 200
for i in range(steps):

    # Load derivatives Tint wrt Age and Mass
    data = np.genfromtxt(path_der + "derivativeTint_" + ex + "_N%i_sigma%.1f_v%i.dat"
                    %(nBDs, sigma, i+1), unpack=True)

    dervTint_A = data[0]
    dervTint_M = data[1]

## calculate predictic intrinsic temperature
xi       = np.transpose(np.asarray([Aobs, Mobs]))
Teff     = griddata(points, values, xi)


Mock observations took 65.52050709724426


In [32]:
conv_Msun_to_kg         = 1.98841e+30 # [kg/Msun]
from derivatives import derivativeTDM_wrt_M, derivativeTDM_wrt_r
from utils import T_DM, temperature_withDM
import time

def sigma_Tmodel2(r, M, A, sigma_r, sigma_M, sigma_A,
                  Tint, points, values, dervTint_M, dervTint_A, f, params,
                  v=None, R=R_jup.value, Rsun=8.178, epsilon=1):
    """
    Return squared uncertainty in model temperature [UNITS??]

    Input:
        r : Galactocentric distance [kpc]
        M : mass [Msun]
        A : age [Gyr]

    Assumption: uncertainties in age, mass and galactocentric distance
        are independent
    """
    M_in_kg = M*conv_Msun_to_kg

    _TDM = T_DM(r, R=R, M=M_in_kg, Rsun=Rsun, f=f, params=params, v=v,
                epsilon=epsilon)
    Ttot = np.power(_TDM**4 + Tint**4, 0.25)

    dervT_M = ((Tint/Ttot)**3*dervTint_M +
               (_TDM/Ttot)**3*derivativeTDM_wrt_M(r, f, params, M, v=v, R=R,
                                                  Rsun=Rsun,epsilon=epsilon))
    
    return (np.power((Tint/Ttot)**3*dervTint_A*sigma_A, 2)+
            np.power(dervT_M*sigma_M, 2)+
            np.power((_TDM/Ttot)**3*derivativeTDM_wrt_r(r, f, params, M, v=v,
                                  R=R, Rsun=Rsun, epsilon=epsilon)*sigma_r, 2))

    # result = ((Tint/Ttot)**3*dervTint_A*sigma_A)**2 + (dervT_M*sigma_M)**2 + ((_TDM/Ttot)**3*derivativeTDM_wrt_r(r, f, params, M, v=v,
    #                               R=R, Rsun=Rsun, epsilon=epsilon)*sigma_r)**2
    # return result

In [33]:

timers = []
for i in range(150):
    start = time.time()
    _sigma_Tmodel2 = sigma_Tmodel2(robs, Mobs, Aobs, sigmarobs, sigmaMobs,
                                        sigmaAobs, Teff, points, values, dervTint_M,
                                        dervTint_A,
                                        f_true, [gamma_true, rs_true, rho0], v=v, R=R_jup.value, Rsun=Rsun,
                                        epsilon=epsilon)
    elapsed = time.time() - start
    timers.append(elapsed)

print("Input section took " + str(np.mean(timers)))

Input section took 0.14516403516133627


In [31]:

timers = []
for i in range(150):
    start = time.time()
    _sigma_Tmodel2 = sigma_Tmodel2(robs, Mobs, Aobs, sigmarobs, sigmaMobs,
                                        sigmaAobs, Teff, points, values, dervTint_M,
                                        dervTint_A,
                                        f_true, [gamma_true, rs_true, rho0], v=v, R=R_jup.value, Rsun=Rsun,
                                        epsilon=epsilon)
    elapsed = time.time() - start
    timers.append(elapsed)

print("Input section took " + str(np.mean(timers)))

Input section took 0.14202037811279297


## Using Numba

In [6]:
from astropy.constants import R_jup, M_jup, G, sigma_sb
import numpy as np
def gNFW_rho(Rsun, R, parameters):
    """
    Return gNFW density profile at r distance from the GC
    Denstiy has same units as local DM density rho0
    """
    # gNFW parameters
    gamma = parameters[0] 
    Rs    = parameters[1]
    rho0  = parameters[2]
    # Density profile
    rho   = rho0*np.power(Rsun/R, gamma)*np.power((Rs+Rsun)/(Rs+R), 3-gamma)    
    # Return
    return rho

In [7]:
Rsun                    = 8.178
rho0                    = 0.42
rs = 5
gamma = 0.6
_gNFW_rho = gNFW_rho(Rsun, R_jup.value, [gamma, rs, rho0])